In [ ]:
%%configure -f
{"defaultLakehouse": {"name": "lh_cfg"}}

In [ ]:
# Upload file "configuration.xlsx" to "lh_cfg/Files"
# Edit variable "prj_doc_data_file_name"

# Imports

In [ ]:
import sempy.fabric                       as f
from   pyspark.sql       import functions as sf
import pandas                             as pd
from   pyspark.sql.types import *       # as st

# Settings

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

In [ ]:
is_debug               = True
prj_doc_data_file_name = "configuration" # Edit this manually

In [ ]:
# Get current WorkspaceID
workspace_id = f.get_workspace_id()
if is_debug: print(f"workspace_id: {workspace_id}")

# Code

## fn_exec_spark_sql

In [ ]:
def fn_exec_spark_sql(sql_code):
  try:
    return spark.sql(sql_code)
  except Exception (ex):
    return str(ex)

## Get the ABFS Paths

In [ ]:
# Create variables for abfs paths (abfs_path_<lakehouse_name>)
list_lakehouse = mssparkutils.lakehouse.list(workspaceId = workspace_id)

for v in list_lakehouse:
    lakehouse_name = v.displayName
    abfs_path = v.properties["abfsPath"]

    if is_debug: print(f"abfs_path_{lakehouse_name}: {abfs_path}")
    locals()[f"abfs_path_{lakehouse_name}"] = abfs_path

##  fn_cleanup_sdf

In [ ]:
# Replace NaN and empty string with NULL
def fn_cleanup_sdf(sdf):
  cols = sdf.columns
  for c in cols: sdf = sdf.withColumn(c, sf.when((sf.col(c) == "NaN"), None).otherwise(sf.col(c)))
  for c in cols: sdf = sdf.withColumn(c, sf.when((sf.col(c) == ""   ), None).otherwise(sf.col(c)))

  return sdf

## Get Excel file name and sheet names

In [ ]:
file_name       = f"{abfs_path_lh_cfg}/Files/{prj_doc_data_file_name}.xlsx"
sheet_name_list = pd.ExcelFile(f"{abfs_path_lh_cfg}/Files/{prj_doc_data_file_name}.xlsx").sheet_names
if is_debug:
    print(f"file_name: {file_name}")
    display(f"sheet_name_list: {sheet_name_list}")

## Append to table 'lh_cfg.global_parameter'

In [ ]:
sheet_name = "Extract Global Parameters"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[1]].count()

  if is_debug: print(f"pdf_count: {pdf_count}")

  if pdf_count > 0:
    sdf_new = spark.createDataFrame(pdf)

    sdf_new = sdf_new.select(
      sf.trim(sdf_new["name"])   .alias("name") .cast(StringType())
      , sf.trim(sdf_new["value"]).alias("value").cast(StringType()) 
      , sf.trim(sdf_new["note"]) .alias("note") .cast(StringType())
    )

    sdf_new = fn_cleanup_sdf(sdf_new)
    sdf_new = sdf_new.na.drop(subset = ["name"])

    sdf_exst = fn_exec_spark_sql(f"""
      SELECT
        TRIM(`name`)  AS `name`
        , TRIM(`value`) AS `value`
        , TRIM(`note`)  AS `note`
      FROM delta.`{abfs_path_lh_cfg}/Tables/global_parameter`
    """)

    sdf_new.createOrReplaceTempView("tvw_new")
    sdf_exst.createOrReplaceTempView("tvw_existing")

    sdf = fn_exec_spark_sql("""
      SELECT
        TRIM(N.`name`) AS `name`
        , TRIM(
          CASE
            WHEN N.`value` = 'Oui' THEN '1'
            WHEN N.`value` = 'Non' THEN '0'
            ELSE N.`value`
          END)         AS `value`
        , TRIM(N.`note`) AS `note`
      FROM
        tvw_new                AS N
        LEFT JOIN tvw_existing AS E ON N.`name` = E.`name`
      WHERE E.`name` IS NULL
    """)

    sdf_count = sdf.count()

    if sdf_count > 0:
      sdf.write\
        .format("delta")\
        .mode("append")\
        .save(f"{abfs_path_lh_cfg}/Tables/global_parameter")

    if is_debug:
      sdf_new.printSchema()
      display(sdf_new)
      
      sdf_exst.printSchema()
      display(sdf_exst)
      
      sdf.printSchema()
      display(sdf)

      print(f"sdf_count: {str(sdf_count)}")

## Insert into table 'lh_cfg.metadata_column'

In [ ]:
sheet_name = "Extract Metatada Columns"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[1]].count()

  if is_debug: print(f"pdf_count: {pdf_count}")

  if pdf_count > 0:
    sdf = spark.createDataFrame(pdf)

    sdf = sdf.select(
      sf.trim(sdf["Technologie"]).alias("technology").cast(StringType())
      , sf.trim(sdf["Fréquence"]).alias("frequency") .cast(StringType())
      , sf.trim(sdf["name"])     .alias("name")      .cast(StringType())
      , sf.trim(sdf["note"])     .alias("note")      .cast(StringType())
    )

    sdf = fn_cleanup_sdf(sdf)
    sdf = sdf.na.drop(subset = ["technology"])

    sdf.write\
      .format("delta")\
      .mode("overwrite")\
      .option("overwriteSchema", "True")\
      .save(f"{abfs_path_lh_cfg}/Tables/metadata_column")

    if is_debug:
      sdf.printSchema()
      display(sdf)

## Insert into table "lh_cfg.extract_object_sql_server"

In [ ]:
sheet_name = "Bronze (SQL Server)"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[1]].count()

  if is_debug:
    print(f"pdf_count: {pdf_count}")

  if pdf_count > 0:
    sdf = spark.createDataFrame(pdf)

    sdf = sdf.select(
      sf.lit("SQL Server")                                             .alias("technology")                   .cast(StringType())
      , sf.col("Fréquence")                                            .alias("frequency")                    .cast(StringType())
      , sf.col("Key-vault URL")                                        .alias("keyvault_url")                 .cast(StringType())
      , sf.col("Key-vault Utilisateur")                                .alias("keyvault_secret_name_user")    .cast(StringType())
      , sf.col("Key-vault Mot de passe")                               .alias("keyvault_secret_name_password").cast(StringType())
      , sf.col("Serveur")                                              .alias("server_name")                  .cast(StringType())
      , sf.col("Base de données")                                      .alias("database_name")                .cast(StringType())
      , sf.col("Schéma")                                               .alias("schema_name")                  .cast(StringType())
      , sf.col("Table")                                                .alias("table_name")                   .cast(StringType())
      , sf.col("Type Chargement")                                      .alias("extract_type")                 .cast(StringType())
      , sf.col("Séquence")                                             .alias("sequence")                     .cast(LongType())
      , sf.col("ExtractionTimeframe")                                  .alias("extraction_timeframe")         .cast(StringType())
      , sf.col("FROM")                                                 .alias("from")                         .cast(StringType())
      , sf.col("WHERE")                                                .alias("where")                        .cast(StringType())
      , sf.expr("CASE WHEN `Est PK?` = 'Oui' THEN True ELSE False END").alias("is_pk")                        .cast(BooleanType())
      , sf.col("Préfixe SELECT")                                       .alias("prefix_select")                .cast(StringType())
      , sf.col("Colonne")                                              .alias("column_name")                  .cast(StringType())
      , sf.expr("CASE WHEN Inclure = 'Oui' THEN True ELSE False END")  .alias("is_extracted")                 .cast(BooleanType())
      , sf.col("Type de données")                                      .alias("data_type")                    .cast(StringType())
    )

    sdf = fn_cleanup_sdf(sdf)
    
    sdf.write\
      .format("delta")\
      .mode("overwrite")\
      .option("overwriteSchema", "True")\
      .save(f"{abfs_path_lh_cfg}/Tables/extract_object_sql_server")

    if is_debug:
      sdf.printSchema()
      display(sdf)

## Insert into table "lh_cfg.extract_object_lakehouse"

In [ ]:
sheet_name = "Bronze (Lakehouse)"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[1]].count()

  if is_debug:
    print(f"pdf_count: {pdf_count}")

  if pdf_count > 0:
    sdf = spark.createDataFrame(pdf)

    sdf = sdf.select(
      sf.lit("Lakehouse")                                              .alias("technology")          .cast(StringType())
      , sf.col("Fréquence")                                            .alias("frequency")           .cast(StringType())
      , sf.col("Lakehouse")                                            .alias("lakehouse_name")      .cast(StringType())
      , sf.col("Table")                                                .alias("table_name")          .cast(StringType())
      , sf.col("Type Chargement")                                      .alias("extract_type")        .cast(StringType())
      , sf.col("Séquence")                                             .alias("sequence")            .cast(LongType())
      , sf.col("ExtractionTimeframe")                                  .alias("extraction_timeframe").cast(StringType())
      , sf.col("FROM")                                                 .alias("from")                .cast(StringType())
      , sf.col("WHERE")                                                .alias("where")               .cast(StringType())
      , sf.expr("CASE WHEN `Est PK?` = 'Oui' THEN True ELSE False END").alias("is_pk")               .cast(BooleanType())
      , sf.col("Préfixe SELECT")                                       .alias("prefix_select")       .cast(StringType())
      , sf.col("Colonne")                                              .alias("column_name")         .cast(StringType())
      , sf.expr("CASE WHEN Inclure = 'Oui' THEN True ELSE False END")  .alias("is_extracted")        .cast(BooleanType())
      , sf.col("Type de données")                                      .alias("data_type")           .cast(StringType())
    )

    sdf = fn_cleanup_sdf(sdf)
    
    sdf.write\
      .format("delta")\
      .mode("overwrite")\
      .option("overwriteSchema", "True")\
      .save(f"{abfs_path_lh_cfg}/Tables/extract_object_lakehouse")

    if is_debug:
      sdf.printSchema()
      display(sdf)

## Insert into table "lh_cfg.extract_object_excel"

In [ ]:
sheet_name = "Bronze (Excel)"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[1]].count()

  if is_debug:
    print(f"pdf_count: {pdf_count}")

  if pdf_count > 0:
    sdf = spark.createDataFrame(pdf)

    sdf = sdf.select(
      sf.lit("Excel")                                                  .alias("technology")    .cast(StringType())
      , sf.col("Fréquence")                                            .alias("frequency")     .cast(StringType())
      , sf.col("Répertoire")                                           .alias("folder_name")   .cast(StringType())
      , sf.col("Fichier")                                              .alias("file_name")     .cast(StringType())
      , sf.col("Feuille de travail")                                   .alias("worksheet_name").cast(StringType())
      , sf.col("Type Chargement")                                      .alias("extract_type")  .cast(StringType())
      , sf.col("Première rangée")                                      .alias("first_row")     .cast(StringType())
      , sf.col("Séquence")                                             .alias("sequence")      .cast(LongType())
      , sf.expr("CASE WHEN `Est PK?` = 'Oui' THEN True ELSE False END").alias("is_pk")         .cast(BooleanType())
      , sf.col("Colonne")                                              .alias("column_name")   .cast(StringType())
      , sf.expr("CASE WHEN Inclure = 'Oui' THEN True ELSE False END")  .alias("is_extracted")  .cast(BooleanType())
      , sf.col("Type de données")                                      .alias("data_type")     .cast(StringType())
    )

    sdf = fn_cleanup_sdf(sdf)
    
    sdf.write\
      .format("delta")\
      .mode("overwrite")\
      .option("overwriteSchema", "True")\
      .save(f"{abfs_path_lh_cfg}/Tables/extract_object_excel")

    if is_debug:
      sdf.printSchema()
      display(sdf)

## Insert into table "lh_cfg.extract_object_csv"

In [ ]:
sheet_name = "Bronze (CSV)"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[2]].count()

  if is_debug:
    print(f"pdf_count:  {pdf_count}")

  if pdf_count > 0:
    sdf = spark.createDataFrame(pdf)

    sdf = sdf.select(      
      sf.lit("CSV")                                                          .alias("technology")   .cast(StringType())
      , sf.col("Fréquence")                                                  .alias("frequency")    .cast(StringType())
      , sf.col("Répertoire")                                                 .alias("folder_name")  .cast(StringType())
      , sf.col("Fichier")                                                    .alias("file_name")    .cast(StringType())
      , sf.expr("CASE WHEN `A un en-tête?` = 'Oui' THEN True ELSE False END").alias("has_header")   .cast(BooleanType())
      , sf.col("Délimiteur")                                                 .alias("delimiter")    .cast(StringType())
      , sf.col("Séparateur de lignes")                                       .alias("row_separator").cast(StringType())
      , sf.col("Citation")                                                   .alias("quote")        .cast(StringType())
      , sf.col("Type Chargement")                                            .alias("extract_type") .cast(StringType())
      , sf.col("Séquence")                                                   .alias("sequence")     .cast(LongType())
      , sf.expr("CASE WHEN `Est PK?` = 'Oui' THEN True ELSE False END")      .alias("is_pk")        .cast(BooleanType())
      , sf.col("Colonne")                                                    .alias("column_name")  .cast(StringType())
      , sf.expr("CASE WHEN Inclure = 'Oui' THEN True ELSE False END")        .alias("is_extracted") .cast(BooleanType())
      , sf.col("Type de données")                                            .alias("data_type")    .cast(StringType())
    )

    sdf = fn_cleanup_sdf(sdf)
    
    sdf.write\
      .format("delta")\
      .mode("overwrite")\
      .option("overwriteSchema", "True")\
      .save(f"{abfs_path_lh_cfg}/Tables/extract_object_csv")

    if is_debug:
      sdf.printSchema()
      display(sdf)

## Insert into table "lh_cfg.extract_object_json"

In [ ]:
sheet_name = "Bronze (JSON)"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[2]].count()

  if is_debug:
    print(f"pdf_count: {pdf_count}")

  if pdf_count > 0:
    sdf = spark.createDataFrame(pdf)

    sdf = sdf.select(
      sf.lit("JSON")                                                   .alias("technology")        .cast(StringType())
      , sf.col("Fréquence")                                            .alias("frequency")         .cast(StringType())
      , sf.col("Répertoire")                                           .alias("folder_name")       .cast(StringType())
      , sf.col("Fichier")                                              .alias("file_name")         .cast(StringType())
      , sf.col("Type Chargement")                                      .alias("extract_type")      .cast(StringType())
      , sf.col("Séquence")                                             .alias("sequence")          .cast(LongType())
      , sf.expr("CASE WHEN `Est PK?` = 'Oui' THEN True ELSE False END").alias("is_pk")             .cast(BooleanType())
      , sf.col("Colonne")                                              .alias("column_name")       .cast(StringType())
      , sf.col("Colonne parent")                                       .alias("parent_column_name").cast(StringType())
      , sf.expr("CASE WHEN Inclure = 'Oui' THEN True ELSE False END")  .alias("is_extracted")      .cast(BooleanType())
      , sf.col("Type de données")                                      .alias("data_type")         .cast(StringType())
    )

    sdf = fn_cleanup_sdf(sdf)
    
    sdf.write\
      .format("delta")\
      .mode("overwrite")\
      .option("overwriteSchema", "True")\
      .save(f"{abfs_path_lh_cfg}/Tables/extract_object_json")

    if is_debug:
      sdf.printSchema()
      display(sdf)

## Insert into table "lh_cfg.extract_object_api"

In [ ]:
sheet_name = "Bronze (API)"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[2]].count()

  if is_debug:
    print(f"pdf_count: {pdf_count}")

  if pdf_count > 0:
    sdf = spark.createDataFrame(pdf)

    sdf = sdf.select(
        sf.lit("API")                                                  .alias("technology")         .cast(StringType())
        , sf.col("Fréquence")                                          .alias("frequency")          .cast(StringType())
        , sf.col("Table")                                              .alias("table_name")         .cast(StringType())
        , sf.col("Base URL")                                           .alias("base_url")           .cast(StringType())
        , sf.col("Method")                                             .alias("method")             .cast(StringType())
        , sf.col("Header")                                             .alias("header")             .cast(StringType())
        , sf.col("Body")                                               .alias("body")               .cast(StringType())
        , sf.col("Root Element")                                       .alias("root_element_name")  .cast(StringType())
        , sf.col("Pagination Value")                                   .alias("paginated_value")    .cast(StringType())
        , sf.col("Parameter")                                          .alias("params")             .cast(StringType()) 
        , sf.col("Authorization Type")                                 .alias("authorization_type") .cast(StringType())
        , sf.col("Authorization Value")                                .alias("authorization_value").cast(StringType())
        , sf.col("Request Timeout")                                    .alias("request_timeout")    .cast(IntegerType())
        , sf.col("Response Format")                                    .alias("response_format")    .cast(StringType())
        , sf.expr("CASE WHEN Inclure = 'Oui' THEN True ELSE False END").alias("is_extracted")       .cast(BooleanType())      
    )

    sdf = fn_cleanup_sdf(sdf)
    
    sdf.write\
      .format("delta")\
      .mode("overwrite")\
      .option("overwriteSchema", "True")\
      .save(f"{abfs_path_lh_cfg}/Tables/extract_object_api")

    if is_debug:
      sdf.printSchema()
      display(sdf)

## Insert into table "lh_cfg.power_bi_refresh"

In [ ]:
sheet_name = "Power BI Refresh"
if is_debug: print(f"sheet_name: {sheet_name}")

if sheet_name in sheet_name_list:
  pdf = pd.read_excel(
    io         = file_name,
    sheet_name = sheet_name,
    header     = 1
  )

  pdf_count = pdf[pdf.columns[2]].count()

  if is_debug:
    print(f"pdf_count: {pdf_count}")

  if pdf_count > 0:
    sdf = spark.createDataFrame(pdf)

    sdf = sdf.select(
      sf.col("Fréquence")                                            .alias("frequency")             .cast(StringType())
      , sf.col("Environnement")                                      .alias("environment")           .cast(StringType())
      , sf.col("Workspace ID")                                       .alias("workspace_id")          .cast(StringType())
      , sf.col("Semantic Model ID")                                  .alias("semantic_model_id")     .cast(StringType())
      , sf.col("Nom Semantic Model")                                 .alias("semantic_model_name")   .cast(StringType())
      , sf.col("Request Timeout")                                    .alias("request_timeout")       .cast(IntegerType())
      , sf.col("Refresh Timeout")                                    .alias("refresh_timeout")       .cast(IntegerType())
      , sf.col("Attendre (secondes)")                                .alias("loop_wait_time_seconds").cast(IntegerType())        
      , sf.expr("CASE WHEN Inclure = 'Oui' THEN True ELSE False END").alias("is_active")             .cast(BooleanType())      
    )

    sdf = fn_cleanup_sdf(sdf)
    
    sdf.write\
      .format("delta")\
      .mode("overwrite")\
      .option("overwriteSchema", "True")\
      .save(f"{abfs_path_lh_cfg}/Tables/power_bi_refresh")

    if is_debug:
      sdf.printSchema()
      display(sdf)

## 